In [10]:
import numpy as np 
import pandas as pd
import sklearn 
import tensorflow as tf 
import nltk
import gensim
import json

### Load Data

In [11]:
##TO-DO: Read dataset.json as a dictionary object 
with open('dataset.json', 'r') as file:
    json_str = file.read()
    json_dic = json.loads(json_str)

In [12]:
##TO-DO: Convert dictionary object to pandas dataframe 
df = pd.DataFrame.from_dict(json_dic).T
for i in range (len(df.index)):
    df['post_tokens'].iloc[i] = " ".join(df['post_tokens'].iloc[i])
df.head()

,post_id,annotators,rationales,post_tokens
1179055004553900032_twitter,1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],i dont think im getting my baby them white 9 h...
1179063826874032128_twitter,1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],we cannot continue calling ourselves feminists...
1178793830532956161_twitter,1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],nawt yall niggers ignoring me
1179088797964763136_twitter,1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",<user> i am bit confused coz chinese ppl can n...
1179085312976445440_twitter,1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",this bitch in whataburger eating a burger with...


In [13]:
from nltk.tokenize import word_tokenize

nltk.download('punkt')

df['tokenized'] = df.apply(lambda row: nltk.word_tokenize(row['post_tokens']), axis=1)

print(df['tokenized'])




[nltk_data] Downloading package punkt to /Users/23brianc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1179055004553900032_twitter    [i, dont, think, im, getting, my, baby, them, ...
1179063826874032128_twitter    [we, can, not, continue, calling, ourselves, f...
1178793830532956161_twitter                  [nawt, yall, niggers, ignoring, me]
1179088797964763136_twitter    [<, user, >, i, am, bit, confused, coz, chines...
1179085312976445440_twitter    [this, bitch, in, whataburger, eating, a, burg...
                                                     ...                        
9989999_gab                    [if, ur, still, on, twitter, tell, carlton, i,...
9990225_gab                    [when, i, first, got, on, here, and, said, i, ...
9991681_gab                    [was, macht, der, moslem, wenn, der, zion, geg...
9992513_gab                    [it, is, awful, look, at, world, demographics,...
9998729_gab                    [the, jewish, globalist, elite, have, only, im...
Name: tokenized, Length: 20148, dtype: object


### Pre-Process Data

In [14]:
from itertools import chain
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_text(text_features):
    
    text_features = text_features.str.lower()

    stop_words = stopwords.words('english')
    
    # df['tweet_without_stopword'] = text_features.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    df['tweet_without_stopword'] = text_features.apply(lambda x:[word for word in x.split() if word not in (stop_words)])

    tweets_without_stopword = list(df['tweet_without_stopword'])

    lemmatized = []
    
    for words in tweets_without_stopword: #lemmatize each word in df['tweet_without_stopword'] and create new column
        temp = []

        for w in words:
            temp.append(lemmatizer.lemmatize(w))
            
        # lemmatized.append(' '.join(temp))
        lemmatized.append(temp)
        
    df['text_lemmatized'] = lemmatized
    
    """
    pre-process text features of our dataset:  
    - Case Lowering
    - Tokenization
    - Stop Word Removal
    - Lemmatization 
 
    Paramaters
    ---------
    text_features: text (tweet) data in our dataframe.
    """

    ##TO-DO: complete this function!
    ##When determining how to return the processed data, consider the format your data needs to be in for train_test_split and model training

 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/23brianc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/23brianc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
clean_text(df['post_tokens'])

df.head()

,post_id,annotators,rationales,post_tokens,tokenized,tweet_without_stopword,text_lemmatized
1179055004553900032_twitter,1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],i dont think im getting my baby them white 9 h...,"[i, dont, think, im, getting, my, baby, them, ...","[dont, think, im, getting, baby, white, 9, two...","[dont, think, im, getting, baby, white, 9, two..."
1179063826874032128_twitter,1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],we cannot continue calling ourselves feminists...,"[we, can, not, continue, calling, ourselves, f...","[cannot, continue, calling, feminists, rights,...","[cannot, continue, calling, feminist, right, w..."
1178793830532956161_twitter,1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],nawt yall niggers ignoring me,"[nawt, yall, niggers, ignoring, me]","[nawt, yall, niggers, ignoring]","[nawt, yall, nigger, ignoring]"
1179088797964763136_twitter,1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",<user> i am bit confused coz chinese ppl can n...,"[<, user, >, i, am, bit, confused, coz, chines...","[<user>, bit, confused, coz, chinese, ppl, acc...","[<user>, bit, confused, coz, chinese, ppl, acc..."
1179085312976445440_twitter,1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",this bitch in whataburger eating a burger with...,"[this, bitch, in, whataburger, eating, a, burg...","[bitch, whataburger, eating, burger, top, bun,...","[bitch, whataburger, eating, burger, top, bun,..."


In [16]:
print(len(df))

20148


In [7]:
from gensim.models import Word2Vec

def vectorize_text(cleaned_text, use_word2vec=True):
    
    model = Word2Vec(cleaned_text, min_count=1)
    
    feature_matrix = []
    
    for sentence in cleaned_text:
        
        vectors = []
        num_words = len(sentence)
        
        for word in sentence:
            vector = model.wv[word].tolist()
            vectors.append(vector)
        
        averaged_vector = []
        
        for i in range (0, len(vectors[0])):
            sum = 0
            
            for v in vectors:
                sum = sum + v[i]
                
            avg = sum/num_words
            
            averaged_vector.append(avg)
            
        feature_matrix.append(averaged_vector)
        
    return feature_matrix


    """
    use Word2Vec or Sentence2Vec to vectorize the cleaned text data
 
    Paramaters
    ---------
    cleaned_text: (cleaned) text (tweet) data in our dataframe.
    """
    ##TO-DO: complete this function!
    ##When determining how to return the processed data, consider the format your data needs to be in for train_test_split and model training


In [8]:
clean_text(df['post_tokens'])

feature_matrix = vectorize_text(df['text_lemmatized'])

print(len(feature_matrix))
print (len(feature_matrix[3]))

20148
100


In [17]:
feature_matrix

[[-0.31232322781131816,
  0.5855228448143373,
  0.055160816185749494,
  -0.3099472290621354,
  0.6720500084070059,
  -1.1352082840525186,
  0.13990357819084936,
  1.5928352521016047,
  -0.29294433552198684,
  -0.6459363933939201,
  -0.3761990504482618,
  -0.7818459673569753,
  -0.21553506286671528,
  -0.08144866171866082,
  0.49988199655826276,
  -0.21935734973074153,
  0.3618824067883767,
  -0.7621494846848341,
  0.24894658060601124,
  -1.0979469269514084,
  0.06468218967067794,
  0.0653034543479757,
  0.6875784067580333,
  -0.516192284054481,
  -0.2583526772661851,
  0.2902996253508788,
  -0.22580468009870786,
  -0.2856515933258029,
  -0.19088411001631847,
  0.4568411362572358,
  0.888283012004999,
  0.13166629186329934,
  -0.12447349032243857,
  -0.4188267314949861,
  -0.1909230501176073,
  0.7439719106142337,
  -0.0190910331125801,
  -0.03278914714554468,
  -0.14969727239356592,
  -0.999681857916025,
  0.5607937843753741,
  -1.020784980402543,
  -0.09423038016001765,
  0.0054050684

### Model Training: KNN/Random Forest/RNNs

In [31]:
import random

zero_count = random.randint(0, 20148)
one_count = 20148 - zero_count
vector = [0]*zero_count + [1]*one_count
random.shuffle(vector)


In [34]:
from sklearn.model_selection import train_test_split

## TO-DO: split our data into train and test data

X_train, X_test, y_train, y_test = train_test_split(feature_matrix, vector, test_size=0.2, random_state=42)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create an instance of the model
model = RandomForestClassifier(n_estimators=100)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7374689826302729


In [40]:
from sklearn.model_selection import cross_val_score

print("Cross Val Score: ",cross_val_score(model,X_train,y_train,cv=5).mean())
print("Train Score: ", round(model.score(X_train,y_train),4))
print("Test Score: ", round(model.score(X_test,y_test),4))

Cross Val Score:  0.7309219982923605
Train Score:  0.9988
Test Score:  0.7375


##### Using KNN
###### Refer to this resource: https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification

In [ ]:
## TO-DO: Determine initial value of parameters (especially K)
k = 0

## TO-DO: determine the best distance computation metric
candidate_metrics = ['euclidean','cosine','manhattan']
best_metric = ''




In [ ]:
from sklearn.neighbors import KNeighborsClassifier

## TO-DO: fit a KNN Classifier with distance metric and K 

In [ ]:
# TO-DO: calculating accuracy
from sklearn.metrics import accuracy_score


In [ ]:
## TO-DO: optimize K with hyperparameter tuning

from sklearn.model_selection import cross_val_score


## TO-DO: Visualize results to determine best K


In [ ]:
## TO-DO: repeat training with optimized K

##### Using XGBoost/Random Forest
###### Refer to this resource: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [22]:
## TO-DO: Train model naively, with default parameters
from sklearn.ensemble import RandomForestClassifier

In [23]:
## TO-DO: Evaluate base model
from sklearn.metrics import accuracy_score



In [24]:
##TO-DO: Hyperparameter Tuning via RandomizedSearch & GridSearch 

## get current parameters and their values 
from pprint import pprint


## Randomized Search 
from sklearn.model_selection import RandomizedSearchCV
random_grid = {}
rf_random = RandomizedSearchCV()

## Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {}
grid_search = GridSearchCV()

TypeError: RandomizedSearchCV.__init__() missing 2 required positional arguments: 'estimator' and 'param_distributions'

##### Using LSTMs
###### Refer to this resource: https://www.youtube.com/watch?v=8HyCNIVRbSU&ab_channel=TheA.I.Hacker-MichaelPhi

In [25]:
from tensorflow.python.keras.layers import LSTM, Activation, Dropout, Dense, Embedding
from tensorflow.python.keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences


In [26]:
## veryyyyyy basic architecture. TO-DO: complete and expand

model = Sequential()
model.add(Embedding())
model.add(LSTM())
model.add(Dense(2, activation='softmax'))

model.compile(loss='',optimizer='', metrics=['accuracy'])

TypeError: Embedding.__init__() missing 2 required positional arguments: 'input_dim' and 'output_dim'

In [ ]:
epochs = 0
batch_size = 0

## TO-DO: determine training parameters and complete call
model.fit()

In [ ]:
## TO-DO: evaluate function 
model.evaluate()

### Using BERT
###### Refer to: http://jalammar.github.io/illustrated-bert/ 
###### Refer to: https://medium.com/@parthdholakiya180/twitter-hate-detection-using-bert-e7682b2d0a0c 

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer